# 数据读取

In [1]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py

import os
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import subgraph, add_self_loops

# === 导入模型结构 ===
from models import GraphContrastiveLearner, augment_graph, summarize_graph


# ===================== 0️⃣ 路径设置 =====================
train_graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/train_graph_with_labelmask.pt"
val_graph_path   = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/val/pyg/graph.pt"
test_graph_path  = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/test/pyg/graph.pt"

save_dir = "/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/model_save/time_morenode_high_similar_high_similar"
os.makedirs(save_dir, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ===================== 1️⃣ 加载图数据 =====================
print("\n========== 加载 train 图（带 label mask） ==========")
train_data = torch.load(train_graph_path)
summarize_graph(train_data)

print("\n========== 加载 val 图 ==========")
val_data = torch.load(val_graph_path)

print("\n========== 加载 test 图 ==========")
test_data = torch.load(test_graph_path)


# ===================== 2️⃣ 对 train_data.x 做 Z-score 标准化 =====================
x_mean = train_data.x.mean(dim=0, keepdim=True)
x_std  = train_data.x.std(dim=0, keepdim=True) + 1e-6
train_data.x = (train_data.x - x_mean) / x_std

print("🎯 已完成 train_data.x 标准化")

x_mean = test_data.x.mean(dim=0, keepdim=True)
x_std  = test_data.x.std(dim=0, keepdim=True) + 1e-6
test_data.x = (test_data.x - x_mean) / x_std

print("🎯 已完成 test_data.x 标准化")

# ===================== 3️⃣ 构造 batch 子图 =====================
batch_size = 1024
num_nodes = train_data.num_nodes

perm = torch.randperm(num_nodes)

batches = []
for i in range(0, num_nodes, batch_size):
    node_idx = perm[i:i + batch_size]

    # ---- 提取子图（必须 CPU）----
    sub_edge_index, _ = subgraph(
        node_idx,
        train_data.edge_index,
        relabel_nodes=True
    )

    sub_x = train_data.x[node_idx]

    # ---------- ⭐ 加入自环 ----------
    sub_edge_index, _ = add_self_loops(
        sub_edge_index, num_nodes=sub_x.size(0)
    )

    sub_data = Data(
        x=sub_x,
        edge_index=sub_edge_index,
    )
    batches.append(sub_data)

print(f"📌 共生成 {len(batches)} 个子图 batch")

loader = DataLoader(batches, batch_size=1, shuffle=True)


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/src

========== 加载 train 图（带 label mask） ==========


/tmp/ipykernel_118056/1724986247.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(train_graph_path)



🧩 Graph Data Summary
📊 节点数量 (num_nodes): 15000
📈 节点特征维度 (num_features): 4
🔗 边数量 (num_edges): 6000000
🔁 自环数量 (self-loops): 0
🎯 标签维度 (y_dim): torch.Size([15000])

🧾 Data对象包含字段: ['train_nolabel_mask', 'train_withlabel_mask', 'x', 'y', 'edge_index']


========== 加载 val 图 ==========


/tmp/ipykernel_118056/1724986247.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_data = torch.load(val_graph_path)



========== 加载 test 图 ==========
🎯 已完成 train_data.x 标准化
🎯 已完成 test_data.x 标准化


/tmp/ipykernel_118056/1724986247.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_data = torch.load(test_graph_path)


📌 共生成 15 个子图 batch


# STFT&CWT 特征变换

## train

In [18]:
import torch
from torch_geometric.data import Data
import numpy as np
import os
import pandas as pd

# ====================== 0. 文件路径 ======================
train_pt_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/train/pyg/train_graph_with_labelmask.pt"
save_csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/train_STFT_features.csv"

# ====================== 1. 加载原始 train.pt ======================
data = torch.load(train_pt_path, map_location="cpu")

# 原始信号： (N,4)
X = data.x.cpu().numpy()      
N = len(X)

# 原始标签：假设 data.y 是 shape (N, )
labels = data.y.cpu().numpy()

print("节点数:", N)
print("原始特征维度:", X.shape)

# ====================== 2. 滑窗参数 ======================
window = 128        
step   = int(window * 0.5)    

# ====================== 3. 对每个窗口进行 STFT + 决定窗口 label ======================
stft_features = []            
window_indices = []          
window_labels  = []           # 新增：窗口最终标签

n_fft = 32
hop = 16
window_fn = torch.hann_window(n_fft)

num_windows = 0

for start in range(0, N - window + 1, step):

    # ---- 取窗口内数据 ----
    chunk = X[start:start + window]        # shape: (window,4)
    chunk_label = labels[start:start + window]  # shape: (window,)

    # ---- 统计窗口内最频繁标签 ----
    (unique, counts) = np.unique(chunk_label, return_counts=True)
    win_label = unique[np.argmax(counts)]

    window_feat = []                      

    # ---- 对每通道做 STFT ----
    for ch in range(4):
        sig = chunk[:, ch]                
        xt = torch.tensor(sig)

        stft_complex = torch.stft(
            xt,
            n_fft=n_fft,
            hop_length=hop,
            win_length=n_fft,
            window=window_fn,
            return_complex=True
        )

        stft_mag = stft_complex.abs().numpy()    
        stft_mean = stft_mag.mean(axis=1)        

        window_feat.extend(stft_mean.tolist())

    # ---- 保存结果 ----
    stft_features.append(window_feat)
    window_indices.append(start)
    window_labels.append(win_label)
    num_windows += 1


# ====================== 4. 转为 DataFrame 并保存 CSV ======================
stft_features = np.array(stft_features)    
freq_dim = stft_features.shape[1] // 4

print("窗口数量:", num_windows)
print("每窗口 STFT 特征维度:", stft_features.shape[1])

# 构建列名
columns = []
for ch in range(4):
    for f in range(freq_dim):
        columns.append(f"ch{ch+1}_freq{f}")

df = pd.DataFrame(stft_features, columns=columns)

df.insert(0, "window_start", window_indices)   # 记录窗口起点
df["label"] = window_labels                    # 新增：窗口标签

df.to_csv(save_csv_path, index=False)

print(f"✅ 已保存 STFT+标签 特征 CSV：{save_csv_path}")


/tmp/ipykernel_122805/1817627114.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(train_pt_path, map_location="cpu")


节点数: 15000
原始特征维度: (15000, 4)
窗口数量: 233
每窗口 STFT 特征维度: 68
✅ 已保存 STFT+标签 特征 CSV：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/train_STFT_features.csv


## test

In [19]:
import torch
from torch_geometric.data import Data
import numpy as np
import os
import pandas as pd

# ====================== 0. 文件路径 ======================
test_graph_path  = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/test/pyg/graph.pt"
save_csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/test_STFT_features.csv"

# ====================== 1. 加载原始 train.pt ======================
data = torch.load(test_graph_path, map_location="cpu")

# 原始信号： (N,4)
X = data.x.cpu().numpy()      
N = len(X)

# 原始标签：假设 data.y 是 shape (N, )
labels = data.y.cpu().numpy()

print("节点数:", N)
print("原始特征维度:", X.shape)

# ====================== 2. 滑窗参数 ======================
window = 128        
step   = int(window * 0.5)    

# ====================== 3. 对每个窗口进行 STFT + 决定窗口 label ======================
stft_features = []            
window_indices = []          
window_labels  = []           # 新增：窗口最终标签

n_fft = 32
hop = 16
window_fn = torch.hann_window(n_fft)

num_windows = 0

for start in range(0, N - window + 1, step):

    # ---- 取窗口内数据 ----
    chunk = X[start:start + window]        # shape: (window,4)
    chunk_label = labels[start:start + window]  # shape: (window,)

    # ---- 统计窗口内最频繁标签 ----
    (unique, counts) = np.unique(chunk_label, return_counts=True)
    win_label = unique[np.argmax(counts)]

    window_feat = []                      

    # ---- 对每通道做 STFT ----
    for ch in range(4):
        sig = chunk[:, ch]                
        xt = torch.tensor(sig)

        stft_complex = torch.stft(
            xt,
            n_fft=n_fft,
            hop_length=hop,
            win_length=n_fft,
            window=window_fn,
            return_complex=True
        )

        stft_mag = stft_complex.abs().numpy()    
        stft_mean = stft_mag.mean(axis=1)        

        window_feat.extend(stft_mean.tolist())

    # ---- 保存结果 ----
    stft_features.append(window_feat)
    window_indices.append(start)
    window_labels.append(win_label)
    num_windows += 1


# ====================== 4. 转为 DataFrame 并保存 CSV ======================
stft_features = np.array(stft_features)    
freq_dim = stft_features.shape[1] // 4

print("窗口数量:", num_windows)
print("每窗口 STFT 特征维度:", stft_features.shape[1])

# 构建列名
columns = []
for ch in range(4):
    for f in range(freq_dim):
        columns.append(f"ch{ch+1}_freq{f}")

df = pd.DataFrame(stft_features, columns=columns)

df.insert(0, "window_start", window_indices)   # 记录窗口起点
df["label"] = window_labels                    # 新增：窗口标签

df.to_csv(save_csv_path, index=False)

print(f"✅ 已保存 STFT+标签 特征 CSV：{save_csv_path}")


/tmp/ipykernel_122805/3040907526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(test_graph_path, map_location="cpu")


节点数: 5000
原始特征维度: (5000, 4)
窗口数量: 77
每窗口 STFT 特征维度: 68
✅ 已保存 STFT+标签 特征 CSV：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/test_STFT_features.csv


# STFT变化后的train 建图

In [20]:
def build_local_temporal_graph(
    csv_path: str,
    save_dir: str,
    num_edges: int = 10,
    label_col: int = None
):
    """
    基于时间顺序构建局部时序图。
    每一行是一个节点，上下相邻样本构成边。
    标签列可指定索引，若不指定则默认最后一列。
    🚫 自动忽略首列（常用于序号/ID），避免误入特征计算。

    参数:
        csv_path (str): 输入 CSV 文件路径。
        save_dir (str): 图结构文件的保存文件夹。
        num_edges (int): 每个节点的边数（上下平均分配）。
        label_col (int): 标签列索引（默认 None → 最后一列）。
    返回:
        (nodes_csv, edges_csv, graph_pt): 保存的文件路径元组。
    """
    os.makedirs(save_dir, exist_ok=True)

    # === 读取数据 ===
    df = pd.read_csv(csv_path)
    num_nodes = len(df)
    if num_nodes == 0:
        raise ValueError("❌ 输入 CSV 文件为空。")

    # === 标签列判断 ===
    if label_col is None:
        label_col = df.shape[1] - 1

    # === 提取标签列 ===
    y = torch.tensor(df.iloc[:, label_col].values, dtype=torch.long)

    # === 构造特征列（去掉首列 + 标签列）===
    drop_cols = [df.columns[0], df.columns[label_col]] if label_col != 0 else [df.columns[0]]
    df_features = df.drop(columns=drop_cols, errors="ignore")

    # 保留数值列
    df_features = df_features.select_dtypes(include=["float", "int"])
    if df_features.shape[1] == 0:
        raise ValueError("❌ 特征列为空，请检查输入 CSV。")

    # === 构建边 ===
    half = num_edges // 2
    edges = []
    for i in range(num_nodes):
        start_up = max(0, i - half)
        end_down = min(num_nodes, i + half + 1)
        up_neighbors = list(range(start_up, i))
        down_neighbors = list(range(i + 1, end_down))

        total_needed = num_edges
        current = len(up_neighbors) + len(down_neighbors)
        if current < total_needed:
            remaining = total_needed - current
            if i + half + 1 >= num_nodes:  # 下方不够
                extra_up = list(range(max(0, start_up - remaining), start_up))
                up_neighbors = extra_up + up_neighbors
            elif i - half < 0:  # 上方不够
                extra_down = list(range(end_down, min(num_nodes, end_down + remaining)))
                down_neighbors += extra_down

        for j in up_neighbors + down_neighbors:
            edges.append((i, j))
            edges.append((j, i))

    # === 保存节点与边 ===
    nodes_path = os.path.join(save_dir, "nodes.csv")
    edges_path = os.path.join(save_dir, "edges.csv")
    graph_path = os.path.join(save_dir, "graph.pt")

    df_features.to_csv(nodes_path, index=False)
    pd.DataFrame(edges, columns=["source", "target"]).to_csv(edges_path, index=False)

    # === 构建 PyG 图结构 ===
    edge_index = torch.tensor(edges, dtype=torch.long).T
    x = torch.tensor(df_features.values, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index, y=y)

    torch.save(data, graph_path)

    print(f"✅ 图结构构建完成，共 {num_nodes} 个节点，{len(edges)//2} 条无向边")
    print(f"📁 nodes.csv: {nodes_path}")
    print(f"📁 edges.csv: {edges_path}")
    print(f"📁 graph.pt : {graph_path}")
    print(f"🧩 特征维度: {x.shape[1]} (已自动忽略首列与标签列)")

    return nodes_path, edges_path, graph_path



# === 调用本地时序建图函数 ===
# train
csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/train_STFT_features.csv"
pyg_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/train"
num_edges =  5
nodes_csv, edges_csv, graph_pt = build_local_temporal_graph(
    csv_path=csv_path,
    save_dir=pyg_dir,
    num_edges=num_edges
)
# test
csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/test_STFT_features.csv"
pyg_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/test"
num_edges =  5
nodes_csv, edges_csv, graph_pt = build_local_temporal_graph(
    csv_path=csv_path,
    save_dir=pyg_dir,
    num_edges=num_edges
)

✅ 图结构构建完成，共 233 个节点，937 条无向边
📁 nodes.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/train/nodes.csv
📁 edges.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/train/edges.csv
📁 graph.pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/train/graph.pt
🧩 特征维度: 68 (已自动忽略首列与标签列)
✅ 图结构构建完成，共 77 个节点，313 条无向边
📁 nodes.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/test/nodes.csv
📁 edges.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/test/edges.csv
📁 graph.pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/test/graph.pt
🧩 特征维度: 68 (已自动忽略首列与标签列)


# 训练

## 模型初始化

In [23]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py

import os
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import subgraph, add_self_loops

# === 导入模型结构 ===
from models import GraphContrastiveLearner, augment_graph, summarize_graph


# ===================== 0️⃣ 路径设置 =====================
train_graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/train/graph.pt"
# val_graph_path   = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/all_class_morenode_high_similar/val/pyg/graph.pt"
test_graph_path  = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/high_similar/STFT&CWT/STFT_pyg/test/graph.pt"

save_dir = "/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/model_save/time_mn_hs_STFT&CWT"
os.makedirs(save_dir, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ===================== 1️⃣ 加载图数据 =====================
print("\n========== 加载 train 图（带 label mask） ==========")
train_data = torch.load(train_graph_path)
summarize_graph(train_data)

# print("\n========== 加载 val 图 ==========")
# val_data = torch.load(val_graph_path)

print("\n========== 加载 test 图 ==========")
test_data = torch.load(test_graph_path)


# ===================== 2️⃣ 对 train_data.x 做 Z-score 标准化 =====================
x_mean = train_data.x.mean(dim=0, keepdim=True)
x_std  = train_data.x.std(dim=0, keepdim=True) + 1e-6
train_data.x = (train_data.x - x_mean) / x_std

print("🎯 已完成 train_data.x 标准化")

x_mean = test_data.x.mean(dim=0, keepdim=True)
x_std  = test_data.x.std(dim=0, keepdim=True) + 1e-6
test_data.x = (test_data.x - x_mean) / x_std

print("🎯 已完成 test_data.x 标准化")

# ===================== 3️⃣ 构造 batch 子图 =====================
batch_size = 256
num_nodes = train_data.num_nodes

perm = torch.randperm(num_nodes)

batches = []
for i in range(0, num_nodes, batch_size):
    node_idx = perm[i:i + batch_size]

    # ---- 提取子图（必须 CPU）----
    sub_edge_index, _ = subgraph(
        node_idx,
        train_data.edge_index,
        relabel_nodes=True
    )

    sub_x = train_data.x[node_idx]

    # ---------- ⭐ 加入自环 ----------
    sub_edge_index, _ = add_self_loops(
        sub_edge_index, num_nodes=sub_x.size(0)
    )

    sub_data = Data(
        x=sub_x,
        edge_index=sub_edge_index,
    )
    batches.append(sub_data)

print(f"📌 共生成 {len(batches)} 个子图 batch")

loader = DataLoader(batches, batch_size=1, shuffle=True)


# ===================== 4️⃣ 初始化模型 =====================
in_dim = train_data.x.size(1)
hidden_dim = 256
out_dim = 256
proj_dim = 128
tau = 0.5

model = GraphContrastiveLearner(in_dim, hidden_dim, out_dim, proj_dim, tau).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/src

========== 加载 train 图（带 label mask） ==========

🧩 Graph Data Summary
📊 节点数量 (num_nodes): 233
📈 节点特征维度 (num_features): 68
🔗 边数量 (num_edges): 1874
🔁 自环数量 (self-loops): 0
🎯 标签维度 (y_dim): torch.Size([233])

🧾 Data对象包含字段: ['edge_index', 'y', 'x']


========== 加载 test 图 ==========
🎯 已完成 train_data.x 标准化
🎯 已完成 test_data.x 标准化
📌 共生成 1 个子图 batch


/tmp/ipykernel_122805/3949076593.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(train_graph_path)
/tmp/ipykernel_122805/3949076593.py:34: Futur

## GCL训练

In [24]:
# ===================== 5️⃣ Batch 训练 =====================
epochs = 200

best_loss = float("inf")
best_model_path = os.path.join(save_dir, f"KAIST_normal_pretrain_best.pt")

print("\n================= 🚀 开始 Batch 对比学习训练 =================\n")

for epoch in range(1, epochs + 1):
    model.train()
    total_loss = 0.0

    for batch_data in loader:
        batch_data = batch_data[0].to(device)   # batch_size=1 → 取第 0 个

        # ---- 生成两份增强视图 ----
        data1 = augment_graph(
            batch_data,
            feature_drop_prob=0.2,
            edge_drop_prob=0.1,
            noise_std=0.02
        ).to(device)

        data2 = augment_graph(
            batch_data,
            feature_drop_prob=0.2,
            edge_drop_prob=0.1,
            noise_std=0.02
        ).to(device)

        # ---- 计算 InfoNCE 对比损失 ----
        loss = model.compute_loss(
            data1.x, data1.edge_index,
            data2.x, data2.edge_index
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # ======== 日志输出 ========
    if epoch % 10 == 0:
        print(f"Epoch [{epoch:03d}/{epochs}] | InfoNCE Loss: {total_loss:.4f}")

    # ======== 保存当前最优模型 ========
    if total_loss < best_loss:
        best_loss = total_loss

        torch.save({
            "model_state_dict": model.state_dict(),
            "best_loss": best_loss,
            "epoch": epoch,
            "config": {
                "in_dim": in_dim,
                "hidden_dim": hidden_dim,
                "out_dim": out_dim,
                "proj_dim": proj_dim,
                "tau": tau
            }
        }, best_model_path)

        print(f"💾 [BEST MODEL UPDATED] Epoch {epoch} | Loss={total_loss:.4f}")


# ===================== 最终模型保存 =====================
final_path = os.path.join(save_dir, f"KAIST_normal_pretrain_epoch{epochs}.pt")
torch.save({
    "model_state_dict": model.state_dict(),
    "epoch": epochs,
    "config": {
        "in_dim": in_dim,
        "hidden_dim": hidden_dim,
        "out_dim": out_dim,
        "proj_dim": proj_dim,
        "tau": tau
    }
}, final_path)

print(f"\n🏁 最终模型已保存至: {final_path}")
print(f"🏆 最优模型已保存至: {best_model_path} | best_loss={best_loss:.4f}")



================= 🚀 开始 Batch 对比学习训练 =================

💾 [BEST MODEL UPDATED] Epoch 1 | Loss=5.4762
💾 [BEST MODEL UPDATED] Epoch 2 | Loss=5.1485
💾 [BEST MODEL UPDATED] Epoch 3 | Loss=4.9452
💾 [BEST MODEL UPDATED] Epoch 4 | Loss=4.7419
💾 [BEST MODEL UPDATED] Epoch 5 | Loss=4.6765
💾 [BEST MODEL UPDATED] Epoch 6 | Loss=4.6009
💾 [BEST MODEL UPDATED] Epoch 7 | Loss=4.5689
💾 [BEST MODEL UPDATED] Epoch 8 | Loss=4.5287
💾 [BEST MODEL UPDATED] Epoch 9 | Loss=4.4987
Epoch [010/200] | InfoNCE Loss: 4.4824
💾 [BEST MODEL UPDATED] Epoch 10 | Loss=4.4824
💾 [BEST MODEL UPDATED] Epoch 11 | Loss=4.4559
💾 [BEST MODEL UPDATED] Epoch 12 | Loss=4.4457
💾 [BEST MODEL UPDATED] Epoch 13 | Loss=4.4129
💾 [BEST MODEL UPDATED] Epoch 14 | Loss=4.3961
💾 [BEST MODEL UPDATED] Epoch 15 | Loss=4.3916
💾 [BEST MODEL UPDATED] Epoch 16 | Loss=4.3829
💾 [BEST MODEL UPDATED] Epoch 17 | Loss=4.3540
💾 [BEST MODEL UPDATED] Epoch 19 | Loss=4.3348
Epoch [020/200] | InfoNCE Loss: 4.3350
💾 [BEST MODEL UPDATED] Epoch 21 | Loss=4.3072
💾

## 分类头

In [25]:
# ======================================================================
#        ⭐ 单 Cell：使用 全部训练数据 整图训练下游分类器 ⭐
# ======================================================================
%run ../_init_path.py
from models import GraphContrastiveLearner, DownstreamClassifier

import torch
import torch.nn.functional as F
from torch.optim import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ======================================================================
# 1. 从 encoder 提取 h_train
# ======================================================================

train_data = train_data.to(device)
model = model.to(device)
model.eval()

print("🚀 提取训练数据特征（Encoder forward）...")

with torch.no_grad():
    h_train, _ = model(train_data.x, train_data.edge_index)

h_train = h_train.detach()         # [N, out_dim]
y_all = train_data.y               # [N]

# ========== ⭐ 不使用 mask，使用全部训练样本 ==========
h_labeled = h_train
y_labeled = y_all

print(f"📌 使用全部样本进行训练: {h_labeled.shape[0]}")
print(f"📌 表征维度(out_dim): {h_labeled.shape[1]}")

# ======================================================================
# 2. 定义分类头（全图训练）
# ======================================================================

num_features = h_labeled.size(1)
num_classes = int(y_labeled.max().item() + 1)

classifier = DownstreamClassifier(
    in_dim=num_features,
    num_classes=num_classes,
    hidden_dim=128
).to(device)

optimizer = Adam(classifier.parameters(), lr=1e-3, weight_decay=1e-4)

# ======================================================================
# 3. 整图训练
# ======================================================================

epochs = 200
print("\n================== 🔥 开始下游分类训练（整图 + 全部数据） ==================\n")

for epoch in range(1, epochs + 1):

    classifier.train()
    optimizer.zero_grad()

    logits = classifier(h_labeled)
    loss = F.cross_entropy(logits, y_labeled)

    loss.backward()
    optimizer.step()

    if epoch % 5 == 0:
        print(f"Epoch {epoch:03d}/{epochs} | Loss: {loss.item():.4f}")

print("\n🏁 下游线性分类训练完成！")

# ======================================================================
# 4. 保存分类器
# ======================================================================

save_path = "/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/model_save/time_mn_hs_STFT&CWT/downstream/downstream_classifier.pt"

torch.save({
    "state_dict": classifier.state_dict(),
    "in_dim": num_features,
    "num_classes": num_classes
}, save_path)

print(f"💾 下游分类器已保存：{save_path}")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/src
🚀 提取训练数据特征（Encoder forward）...
📌 使用全部样本进行训练: 233
📌 表征维度(out_dim): 256

================== 🔥 开始下游分类训练（整图 + 全部数据） ==================

Epoch 005/200 | Loss: 2.3175
Epoch 010/200 | Loss: 1.8574
Epoch 015/200 | Loss: 1.3839
Epoch 020/200 | Loss: 0.9762
Epoch 025/200 | Loss: 0.6878
Epoch 030/200 | Loss: 0.4962
Epoch 035/200 | Loss: 0.3674
Epoch 040/200 | Loss: 0.2754
Epoch 045/200 | Loss: 0.2065
Epoch 050/200 | Loss: 0.1554
Epoch 055/200 | Loss: 0.1182
Epoch 060/200 | Loss: 0.0911
Epoch 065/200 | Loss: 0.0717
Epoch 070/200 | Loss: 0.0575
Epoch 075/200 | Loss: 0.0470
Epoch 080/200 | Loss: 0.0392
Epoch 085/200 | Loss: 0.0333
Epoch 090/200 | Loss: 0.0287
Epoch 095/200 | Loss: 0.0251
Epoch 100/200 | Loss: 0.0222
Epoch 105/200 | Loss:

# acc测试

## train

In [26]:
# ======================================================================
#    ⭐ 单 Cell：加载上游模型 + 下游分类器，并用 Train 做测试 ⭐
# ======================================================================

import torch
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ====================== 模型路径 ======================
pretrain_path = "/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/model_save/time_mn_hs_STFT&CWT/KAIST_normal_pretrain_best.pt"

classifier_path = "/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/model_save/time_mn_hs_STFT&CWT/downstream/downstream_classifier.pt"

print("📌 模型路径已设置")


# ====================== 加载模型结构 ======================
from models import GraphContrastiveLearner, DownstreamClassifier

# ------- 加载上游 GCL 模型 -------
ckpt = torch.load(pretrain_path, map_location=device)
cfg = ckpt["config"]

model = GraphContrastiveLearner(
    in_dim     = cfg["in_dim"],
    hidden_dim = cfg["hidden_dim"],
    out_dim    = cfg["out_dim"],
    proj_dim   = cfg["proj_dim"],
    tau        = cfg["tau"] if "tau" in cfg else 0.5
).to(device)

model.load_state_dict(ckpt["model_state_dict"])
model.eval()
print("✅ 上游模型加载成功！")


# ------- 加载下游分类器 -------
clf_ckpt = torch.load(classifier_path, map_location=device)
in_dim = clf_ckpt["in_dim"]
num_classes = clf_ckpt["num_classes"]

classifier = DownstreamClassifier(
    in_dim=in_dim,
    num_classes=num_classes,
    hidden_dim=128
).to(device)

classifier.load_state_dict(clf_ckpt["state_dict"])
classifier.eval()
print("✅ 下游分类器加载成功！")


# ====================== 加载 Train 图（你先手动加载 train_data） ======================
train_data = train_data.to(device)

print("📌 Train 图数据已加载至 GPU/CPU")


# ======================================================================
#          ⭐ 1️⃣ 提取 Train 的 GCL encoder 表征 h
# ======================================================================
with torch.no_grad():
    h_train, _ = model(train_data.x, train_data.edge_index)

print("🔧 已提取 h_train")


# ======================================================================
#          ⭐ 2️⃣ Train Accuracy（全部训练节点）
# ======================================================================
y_train = train_data.y
with torch.no_grad():
    logits_train = classifier(h_train)
    pred_train = logits_train.argmax(dim=1)

acc_train = (pred_train == y_train).float().mean().item()

print(f"\n🎯 Train Accuracy (ALL train nodes) = {acc_train*100:.2f}%")


# ======================================================================
#          ⭐ 3️⃣ Train metrics（precision/recall/f1）
# ======================================================================
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

y_true_train = y_train.cpu().numpy()
y_pred_train = pred_train.cpu().numpy()

train_acc  = accuracy_score(y_true_train, y_pred_train)
train_prec = precision_score(y_true_train, y_pred_train, average="macro", zero_division=0)
train_reca = recall_score(y_true_train, y_pred_train, average="macro", zero_division=0)
train_f1   = f1_score(y_true_train, y_pred_train, average="macro", zero_division=0)

print("\n====================== TRAIN METRICS ======================")
print(f"Accuracy : {train_acc*100:.2f}%")
print(f"Precision: {train_prec*100:.2f}%")
print(f"Recall   : {train_reca*100:.2f}%")
print(f"F1-score : {train_f1*100:.2f}%")
print("===========================================================\n")


# ======================================================================
#          ⭐ 4️⃣ 统计训练集的错误样本
# ======================================================================
import numpy as np
from collections import Counter

wrong_train_idx = np.where(y_pred_train != y_true_train)[0]

print("============== TRAIN WRONG PREDICTIONS ==============")
print(f"❌ 错误样本数量：{len(wrong_train_idx)} / {len(y_true_train)}")

max_show = 20
for i in wrong_train_idx[:max_show]:
    print(f"Index {i:4d} | True = {y_true_train[i]} | Pred = {y_pred_train[i]}")
if len(wrong_train_idx) > max_show:
    print(f"... (共 {len(wrong_train_idx)} 个错样本，只显示前 {max_show} 个)")

# 每类错误统计
train_wrong_pairs = [(int(y_true_train[i]), int(y_pred_train[i])) for i in wrong_train_idx]
train_wrong_count = Counter(train_wrong_pairs)

print("\n🔍 Train 每类错误统计：(真标签 -> 预测标签)")
for (t, p), cnt in train_wrong_count.items():
    print(f"  {t} → {p} : {cnt} 次")

print("\n🏁 Train-only 评估完成！")


📌 模型路径已设置
✅ 上游模型加载成功！
✅ 下游分类器加载成功！
📌 Train 图数据已加载至 GPU/CPU
🔧 已提取 h_train

🎯 Train Accuracy (ALL train nodes) = 100.00%

====================== TRAIN METRICS ======================
Accuracy : 100.00%
Precision: 100.00%
Recall   : 100.00%
F1-score : 100.00%

============== TRAIN WRONG PREDICTIONS ==============
❌ 错误样本数量：0 / 233

🔍 Train 每类错误统计：(真标签 -> 预测标签)

🏁 Train-only 评估完成！


/tmp/ipykernel_122805/2087874937.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(pretrain_path, map_location=device)
/tmp/ipykernel_122805/2087874937.

## test

In [27]:
# ======================================================================
#    ⭐ 单 Cell：加载上游模型 + 下游分类器，并计算 Test Accuracy ⭐
# ======================================================================

import torch
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from collections import Counter
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ====================== 模型路径 ======================
pretrain_path = "/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/model_save/time_mn_hs_STFT&CWT/KAIST_normal_pretrain_best.pt"
classifier_path = "/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/GCL_pre_train/KAIST_normal_model_opt/model_save/time_mn_hs_STFT&CWT/downstream/downstream_classifier.pt"

print("📌 模型路径已设置")

# ====================== 加载模型结构 ======================
from models import GraphContrastiveLearner, DownstreamClassifier

ckpt = torch.load(pretrain_path, map_location=device)
cfg = ckpt["config"]

model = GraphContrastiveLearner(
    in_dim     = cfg["in_dim"],
    hidden_dim = cfg["hidden_dim"],
    out_dim    = cfg["out_dim"],
    proj_dim   = cfg["proj_dim"],
    tau        = cfg.get("tau", 0.5)
).to(device)

model.load_state_dict(ckpt["model_state_dict"])
model.eval()
print("✅ 上游模型加载成功！")

# ====================== 加载下游分类器 ======================
clf_ckpt = torch.load(classifier_path, map_location=device)

classifier = DownstreamClassifier(
    in_dim=clf_ckpt["in_dim"],
    num_classes=clf_ckpt["num_classes"],
    hidden_dim=128
).to(device)

classifier.load_state_dict(clf_ckpt["state_dict"])
classifier.eval()
print("✅ 下游分类器加载成功！")

# ====================== 读取 Train / Test 图 ======================
# Train 图（你之前已加载到变量 train_data）
train_data = train_data.to(device)

test_data = test_data.to(device)

print(f"📌 Train/Test 图数据已成功读取！")
print(f"   Train 节点数：{train_data.x.size(0)}")
print(f"   Test  节点数：{test_data.x.size(0)}")

# ======================================================================
#    ⭐ 1️⃣ 提取 Train/Test 的 GCL Encoder 表征 h
# ======================================================================
with torch.no_grad():
    h_train, _ = model(train_data.x, train_data.edge_index)
    h_test,  _ = model(test_data.x,  test_data.edge_index)

print("🔧 已提取 h_train / h_test")

# ======================================================================
#    ⭐ 2️⃣ Test Accuracy
# ======================================================================
with torch.no_grad():
    logits_test = classifier(h_test)
    pred_test = logits_test.argmax(dim=1)

acc_test = (pred_test == test_data.y).float().mean().item()
print(f"\n🎯 Test Accuracy = {acc_test*100:.2f}%")

# ======================================================================
#    ⭐ 3️⃣ Test Metrics（Macro 平均）
# ======================================================================
y_true_test = test_data.y.cpu().numpy()
y_pred_test = pred_test.cpu().numpy()

test_acc  = accuracy_score(y_true_test, y_pred_test)
test_prec = precision_score(y_true_test, y_pred_test, average="macro", zero_division=0)
test_reca = recall_score(y_true_test, y_pred_test, average="macro", zero_division=0)
test_f1   = f1_score(y_true_test, y_pred_test, average="macro", zero_division=0)

print("\n======================= TEST METRICS =======================")
print(f"Accuracy : {test_acc*100:.2f}%")
print(f"Precision: {test_prec*100:.2f}%")
print(f"Recall   : {test_reca*100:.2f}%")
print(f"F1-score : {test_f1*100:.2f}%")
print("===========================================================\n")

# ======================================================================
#    ⭐ 4️⃣ Test 错误样本统计
# ======================================================================
wrong_idx = np.where(y_pred_test != y_true_test)[0]

print("============== TEST WRONG PREDICTIONS ===============")
print(f"❌ 错误样本数量：{len(wrong_idx)} / {len(y_true_test)}")

max_show = 20
for i in wrong_idx[:max_show]:
    print(f"Index {i:4d} | True = {y_true_test[i]} | Pred = {y_pred_test[i]}")
if len(wrong_idx) > max_show:
    print(f"... (共 {len(wrong_idx)} 个错样本，仅显示前 {max_show} 个)")

# 每类错误统计
wrong_pairs = [(int(y_true_test[i]), int(y_pred_test[i])) for i in wrong_idx]
wrong_count = Counter(wrong_pairs)

print("\n🔍 Test 每类错误统计：(真标签 → 预测标签)")
for (t, p), cnt in wrong_count.items():
    print(f"  {t} → {p} : {cnt} 次")

print("\n🏁 Test-only 测试完成！")


📌 模型路径已设置
✅ 上游模型加载成功！
✅ 下游分类器加载成功！
📌 Train/Test 图数据已成功读取！
   Train 节点数：233
   Test  节点数：77
🔧 已提取 h_train / h_test

🎯 Test Accuracy = 57.14%

======================= TEST METRICS =======================
Accuracy : 57.14%
Precision: 58.70%
Recall   : 57.08%
F1-score : 55.97%

============== TEST WRONG PREDICTIONS ===============
❌ 错误样本数量：33 / 77
Index    0 | True = 1 | Pred = 13
Index    1 | True = 1 | Pred = 14
Index    2 | True = 1 | Pred = 13
Index    3 | True = 1 | Pred = 13
Index    4 | True = 1 | Pred = 11
Index    5 | True = 1 | Pred = 13
Index    6 | True = 1 | Pred = 13
Index   14 | True = 1 | Pred = 11
Index   20 | True = 11 | Pred = 12
Index   21 | True = 11 | Pred = 12
Index   22 | True = 11 | Pred = 12
Index   23 | True = 11 | Pred = 12
Index   24 | True = 11 | Pred = 14
Index   25 | True = 11 | Pred = 14
Index   26 | True = 11 | Pred = 14
Index   27 | True = 11 | Pred = 13
Index   28 | True = 11 | Pred = 1
Index   30 | True = 12 | Pred = 11
Index   31 | True = 12 | Pred = 1

/tmp/ipykernel_122805/2730359916.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(pretrain_path, map_location=device)
/tmp/ipykernel_122805/2730359916.